In [1]:
from collections import Counter

import pandas as pd

In [2]:
pd.set_option("display.max_columns", None)

In [3]:
df = pd.read_csv("data/df.csv")
df2 = pd.read_csv("data/df2.csv")

In [4]:
df_set = set()
df_set2 = set((row.verb_id, row.complement) for _, row in df2.iterrows())

In [5]:
counter = Counter((row.verb_id, row.complement) for _, row in df.iterrows())

In [6]:
{key: value for key, value in Counter(df.verb_id).items() if value > 1}

{213114: 2,
 1895945: 2,
 225474: 2,
 225482: 2,
 1908906: 2,
 1908914: 2,
 1921795: 2,
 1921803: 2,
 1915493: 2}

In [7]:
{key: value for key, value in Counter(df2.verb_id).items() if value > 1}

{213114: 2, 1895945: 2, 225474: 2, 225482: 2}

In [8]:
df[df.verb_id == 213114]

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,dir_he,dir_he_dss,sign_info,stem,tense,preposition_1,preposition_2,preposition_3,preposition_4
45,213114,BW>[,MT,Isaiah,3,14,JBW>,JHWH B MCPV JBW> <M ZQNJ <MW W FRJW W >TM B<RT...,JHWH B MCPV JBW> <M ZQNJ <MW W FRJW,JHWH,B MCPV,0.0,NaN,NaN,qal,impf,B,NaN,NaN,NaN
46,213114,BW>[,MT,Isaiah,3,14,JBW>,JHWH B MCPV JBW> <M ZQNJ <MW W FRJW W >TM B<RT...,JHWH B MCPV JBW> <M ZQNJ <MW W FRJW,JHWH,<M ZQNJ <MW W FRJW,0.0,NaN,NaN,qal,impf,<M,NaN,NaN,NaN


In [9]:
df2[df2.verb_id == 213114]

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,cmpl_lex,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,sign_info,stem,tense,preposition_1,preposition_2,preposition_3,preposition_4
48,213114,BW>[,MT,Isaiah,3,14,JBW>,JHWH B MCPV JBW> <M ZQNJ <MW W FRJW W >TM B<RT...,JHWH B MCPV JBW> <M ZQNJ <MW W FRJW,JHWH,B MCPV,B MCPV/,0.0,prep,NaN,NaN,und,subs,NaN,qal,impf,B,<M,NaN,NaN
49,213114,BW>[,MT,Isaiah,3,14,JBW>,JHWH B MCPV JBW> <M ZQNJ <MW W FRJW W >TM B<RT...,JHWH B MCPV JBW> <M ZQNJ <MW W FRJW,JHWH,<M ZQNJ <MW W FRJW,<M ZQN/ <M/ W FR/,0.0,prep,NaN,NaN,det,subs subs,NaN,qal,impf,B,<M,NaN,NaN


In [10]:
df_rows_by_verb_id = {}
for _, row in df.iterrows():
    df_rows_by_verb_id.setdefault(row.verb_id, []).append(row)

df2_rows_by_verb_id = {}
for _, row in df2.iterrows():
    df2_rows_by_verb_id.setdefault(row.verb_id, []).append(row)

df_rows_by_complement = {(row.verb_id, row.complement): row for _, row in df.iterrows()}
df2_rows_by_complement = {
    (row.verb_id, row.complement): row for _, row in df2.iterrows()
}

In [11]:
new_columns = [
    "cmpl_lex",
    "cmpl_constr",
    "cmpl_nt",
    "cmpl_anim",
    "cmpl_det",
    "cmpl_indiv",
    "sign_info",
]

In [12]:
# Add columns
items = []
for _, row in df.iterrows():
    similar_rows = df2_rows_by_verb_id[row.verb_id]
    if len(similar_rows) == 1:
        row2 = similar_rows[0]
    else:
        row2 = df2_rows_by_complement[(row.verb_id, row.complement)]
    item = row.to_dict()
    for column in new_columns:
        item[column] = row2[column]
    items.append(item)

# Add rows
for _, row2 in df2.iterrows():
    if row2.verb_id in df_rows_by_verb_id:
        continue
    items.append(row2.to_dict())

merged_df = pd.DataFrame(items)
merged_df

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,dir_he,dir_he_dss,sign_info,stem,tense,preposition_1,preposition_2,preposition_3,preposition_4,cmpl_lex,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv
0,212256,BW>[,MT,Isaiah,1,12,TB>W,KJ TB>W L R>WT PNJ MJ BQC Z>T M JDKM RMS XYRJ,KJ TB>W,NaN,no complement,NaN,NaN,NaN,qal,impf,NaN,NaN,NaN,NaN,no complement,NaN,NaN,NaN,NaN,NaN
1,1895059,BW>[,1Qisaa,Isaiah,1,12,TB>W,KJ> TB>W L R>WT PNJ MJ BQC ZW>T M JDKM L RMWS ...,NaN,NaN,no complement,0.0,0.0,000 0000 0 0000 0000 00 000 0000 0 00000 0 000...,qal,impf,NaN,NaN,NaN,NaN,no complement,NaN,NaN,NaN,NaN,NaN
2,212269,BW>[,MT,Isaiah,1,13,HBJ>,L> TWSJPW HBJ> MNXT CW> QVRT TW<BH HJ> LJ XDC ...,HBJ> MNXT CW>,NaN,no complement,NaN,NaN,NaN,hif,infc,NaN,NaN,NaN,NaN,no complement,NaN,NaN,NaN,NaN,NaN
3,1895075,BW>[,1Qisaa,Isaiah,1,13,HBJ>,LW> TWSJPW L HBJ> MNXT CW> QVRT TW<BH HJ> LJ X...,NaN,NaN,no complement,0.0,0.0,000 000000 0 0000 0000 000 0000 00000 000 000 ...,hifil,infc,NaN,NaN,NaN,NaN,no complement,NaN,NaN,NaN,NaN,NaN
4,212315,SWR[,MT,Isaiah,1,16,HSJRW,RXYW HZKW HSJRW R< M<LLJKM M NGD <JNJ XDLW HR<,HSJRW R< M<LLJKM M NGD <JNJ,NaN,M NGD <JNJ,0.0,NaN,NaN,hif,impv,MN,NaN,NaN,NaN,MN NGD/ <JN/,prep,NaN,NaN,det,subs subs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1399,1926204,XWC[,1Q8,Isaiah,60,22,>XJCNH,H QVN JHJH L >LP W H Y<JR L GWJ <YWM >NJ JHWH ...,>NJ JHWH B <TH >XJCNH,>NJ JHWH,no complement,NaN,NaN,0 000 0000 0 000 0 0 0000 0 000 0000 000 0000 ...,hifil,impf,NaN,NaN,NaN,NaN,no complement,NaN,NaN,NaN,NaN,NaN
1400,1917415,BW>[,1Qisaa,Isaiah,63,4,B>H,KJ> JWM NQM B LBJ W CNT G>WLJ B>H,W CNT G>WLJ B>H,CNT G>WLJ,no complement,NaN,NaN,000 000 000 0 0000 0 000 000000 000 00,qal,perf,NaN,NaN,NaN,NaN,no complement,NaN,NaN,NaN,NaN,NaN
1401,1926538,BW>[,1Q8,Isaiah,63,4,B>H,KJ JWM NQM B LBJ W CNT G>WLJ B>H,W CNT G>WLJ B>H,CNT G>WLJ,no complement,NaN,NaN,00 000 000 0 0000 0 000 000000 000 00,qal,perf,NaN,NaN,NaN,NaN,no complement,NaN,NaN,NaN,NaN,NaN
1402,1918625,CVP[,1Qisaa,Isaiah,66,12,CWVP,KJ> KWH >MR JHWH HNNJ NWVH >LJH> K NHR CLWM W ...,W K NXL CWVP KBWD GW>JM,NaN,no complement,NaN,NaN,000 000 000 0000 00000 0000 000000 0 000 0000 ...,qal,ptca,NaN,NaN,NaN,NaN,no complement,NaN,NaN,NaN,NaN,NaN


In [13]:
merged_df["motion_type"] = ""

In [14]:
merged_df["chapter"] = pd.to_numeric(merged_df["chapter"])
merged_df["verse_num"] = pd.to_numeric(merged_df["verse_num"])

merged_df["dir_he"] = merged_df["dir_he"].astype(str)
merged_df["dir_he"] = merged_df["dir_he"].replace("1.0", "1").replace("0.0", "0")

merged_df["dir_he_dss"] = merged_df["dir_he_dss"].astype(str)
merged_df["dir_he_dss"] = merged_df["dir_he_dss"].replace("1.0", "1").replace("0.0", "0")
merged_df.sort_values(["book", "chapter", "verse_num", "scroll"], ascending=[True, True, True, False], ignore_index=True, inplace=True)

merged_df.fillna("", inplace=True)

In [15]:
merged_df

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,dir_he,dir_he_dss,sign_info,stem,tense,preposition_1,preposition_2,preposition_3,preposition_4,cmpl_lex,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,motion_type
0,212256,BW>[,MT,Isaiah,1,12,TB>W,KJ TB>W L R>WT PNJ MJ BQC Z>T M JDKM RMS XYRJ,KJ TB>W,,no complement,nan,nan,,qal,impf,,,,,no complement,,,,,,
1,1895059,BW>[,1Qisaa,Isaiah,1,12,TB>W,KJ> TB>W L R>WT PNJ MJ BQC ZW>T M JDKM L RMWS ...,,,no complement,0,0,000 0000 0 0000 0000 00 000 0000 0 00000 0 000...,qal,impf,,,,,no complement,,,,,,
2,212269,BW>[,MT,Isaiah,1,13,HBJ>,L> TWSJPW HBJ> MNXT CW> QVRT TW<BH HJ> LJ XDC ...,HBJ> MNXT CW>,,no complement,nan,nan,,hif,infc,,,,,no complement,,,,,,
3,1895075,BW>[,1Qisaa,Isaiah,1,13,HBJ>,LW> TWSJPW L HBJ> MNXT CW> QVRT TW<BH HJ> LJ X...,,,no complement,0,0,000 000000 0 0000 0000 000 0000 00000 000 000 ...,hifil,infc,,,,,no complement,,,,,,
4,212315,SWR[,MT,Isaiah,1,16,HSJRW,RXYW HZKW HSJRW R< M<LLJKM M NGD <JNJ XDLW HR<,HSJRW R< M<LLJKM M NGD <JNJ,,M NGD <JNJ,0,nan,,hif,impv,MN,,,,MN NGD/ <JN/,prep,,,det,subs subs,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1399,1918901,BW>[,1Qisaa,Isaiah,66,23,JBW>,W HJH M DJ XWDC B XWDCW W M DJ CBT B CBTH JBW>...,,,no complement,nan,0,0 000 0 00 0000 0 000000 0 0 00 000 0 00000 00...,qal,impf,,,,,no complement,,,,,,
1400,1927678,BW>[,1Q8,Isaiah,66,23,JBW>,W HJH M DJ XDC B XDCW W M DJ CBT B CBTW JBW> K...,,,no complement,nan,0,0 000 0 00 000 0 00000 0 0 00 000 0 00000 0000...,qal,impf,,,,,no complement,,,,,,
1401,234980,JY>[,MT,Isaiah,66,24,JY>W,W JY>W W R>W B PGRJ H >NCJM H PC<JM BJ KJ TWL<...,W JY>W,,no complement,nan,nan,,qal,perf,,,,,no complement,,,,,,
1402,1918913,JY>[,1Qisaa,Isaiah,66,24,JY>W,W JY>W W R>W B PGRJ H >NCJM H PWC<JM BJ> KJ> T...,,,no complement,nan,0,0 0000 0 000 0 0000 0 00000 0 000000 0000 000 ...,qal,perf,,,,,no complement,,,,,,


In [16]:
merged_df.to_csv("data/full_isaiah_dataset.csv", index=False)